# 自定义层

本节将展示如何构建自定义层

## 1. 不带参数的层

下面的CenteredLayer类要从其输入中减去均值。 要构建它，我们只需继承基础层类并实现前向传播功能。

In [4]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [5]:
'''向该层提供一些数据，验证是否可以按照预期工作'''
layer = CenteredLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32))

tensor([-2., -1.,  0.,  1.,  2.])

In [6]:
'''现在将层作为组件合并到更复杂的模型中'''
net = nn.Sequential(
    nn.Linear(8, 128),
    CenteredLayer()
    )

In [7]:
'''作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。 
由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。
'''
Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.7940e-09, grad_fn=<MeanBackward0>)